# Tamm-Dancoff approximation


The Tamm--Dancoff approximation (TDA) corresponds to setting $\mathbf{B}=\mathbf{0}$ in the electronic Hessian $\mathbf{E}^{[2]}$ and diagonalize only matrix block $\mathbf{A}$.

The `TDAExciDriver` class object implements an iterative reduced-space TDA solver.

In [4]:
import veloxchem as vlx
import numpy as np

mol_str = """
C        0.67759997    0.00000000    0.00000000
C       -0.67759997    0.00000000    0.00000000
H        1.21655197    0.92414474    0.00000000
H        1.21655197   -0.92414474    0.00000000
H       -1.21655197   -0.92414474    0.00000000
H       -1.21655197    0.92414474    0.00000000
"""
molecule = vlx.Molecule.read_str(mol_str, units='angstrom')
basis = vlx.MolecularBasis.read(molecule, "6-31g")

scf_drv = vlx.ScfRestrictedDriver()
scf_drv.compute(molecule, basis)

orbital_energies = scf_drv.scf_tensors['E']
C = scf_drv.scf_tensors['C'] # MO coefficients


method_settings = {} # Hartree-Fock is default

lres_drv = vlx.LinearResponseEigenSolver()
rsp_settings = {'nstates': 1} # the (pi,pi*)-transition
lres_drv.update_settings(rsp_settings, method_settings)

lres_out = lres_drv.compute(molecule, basis, scf_drv.scf_tensors)

tda_drv = vlx.TDAExciDriver()
tda_drv.update_settings(rsp_settings, method_settings)
tda_results = tda_drv.compute(molecule, basis, scf_drv.scf_tensors)
tda_eig_vals = tda_results["eigenvalues"]

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Hartree-Fock                                         
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                

In [5]:
print(f'TDA excitation energy (a.u.):', tda_eig_vals)

TDA excitation energy (a.u.): [0.31143785]


Alternatively, given the small matrix dimension in our numerical example, we can perform a direct matrix diagonalization of the Hermitian $\mathbf{A}$-block with use of NumPy routine `linalg.eigh`.

In [8]:
lres_drv = vlx.LinearResponseEigenSolver()
E2 = lres_drv.get_e2(molecule, basis, scf_drv.scf_tensors)

nocc = molecule.number_of_alpha_electrons()
norb = orbital_energies.shape[0]
nvirt = norb - nocc
n = nocc * nvirt

A = E2[0:n, 0:n]

eigs_TDA, X_TDA = np.linalg.eigh(A)
print(f'TDA excitation energy (a.u.): {np.min(eigs_TDA) : .6f}')

                                                                                                                          
* Info * Processing Fock builds... (batch size: 576)                                                                      
* Info *   batch 1/1                                                                                                      
TDA excitation energy (a.u.):  0.311438
